In [2]:
import numpy as np
import imageio
import random
import os
import matplotlib.pyplot as plt
import glob
import networkx as nx

In [60]:
class randomWalk():
    '''
    Class to generate random walk on the lattice and save agent's path as a video.
    '''
    
    def __init__(self, latticeSize = 20, planeSizeX = 20, planeSizeY = 20):
        '''
        Initializes the lattice and the agent.
        
        Parameters:
        latticeSize - size of the square lattice, the default is 20
        planeSizeX - horizontal size of the plane, the default is 20
        planeSizeY - vertical size of the plane, the default is 20
        '''
        
        self.timeSteps = None
        self.latticeSize = latticeSize
        self.planeSizeX = planeSizeX
        self.planeSizeY = planeSizeY
        self.moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        
        if not os.path.exists('lattice'):
            os.makedirs('lattice')
        if not os.path.exists('plane'):
            os.makedirs('plane')
        if not os.path.exists('randomGraph'):
            os.makedirs('randomGraph')
        
    def latticeARW(self, timeSteps = 100, showTraj = False, toVideo = False):
        '''
        Iterates over every step and generates random walk of the agent and saves results as png file.

        Parameters:
        timeSteps - number of steps done by the agent, the default is 100
        '''
        files = glob.glob(os.path.join('lattice', '*')) # get access to all files in a folder
        for f in files:
            os.remove(f) # remove all files from folder before adding new ones
        
        self.timeSteps = timeSteps

        x = np.zeros(timeSteps + 1)
        y = np.zeros(timeSteps + 1)

        plt.rcParams['figure.figsize'] = [6, 6]
        plt.xlim(-self.latticeSize/2, self.latticeSize/2)
        plt.ylim(-self.latticeSize/2, self.latticeSize/2)
        plt.title("Agent's random walk on a square lattice")

        for t in range(timeSteps):
            nextMove = random.choice(self.moves)
            while abs(x[t] + nextMove[0]) > self.latticeSize/2 or abs(y[t] + nextMove[1]) > self.latticeSize/2:
                nextMove = random.choice(self.moves)
            x[t + 1] = x[t] + nextMove[0]
            y[t + 1] = y[t] + nextMove[1]
            
            if toVideo == True:
                file = os.path.join('lattice', f'move_{t}.png')
                
            if t == 0:
                plt.scatter(x[t], y[t], s = 30, c = 'green')
            if t == timeSteps - 1:
                plt.scatter(x[t], y[t], s = 30, c = 'red')
            plt.scatter(x[t], y[t], s = 3, c = 'gray')
            plt.plot(x[:t+2], y[:t+2], c = 'black', ls = '--', lw = 0.8)
            
            if toVideo == True:
                plt.savefig(file)
            
        if showTraj == True:
            plt.show()
        else:
            plt.close()

            
    def planeARW(self, timeSteps = 100, showTraj = False, toVideo = False):
        '''
        Iterates over every step and generates random walkf of the agent on the plane with randomly chosen angles
        and saves results as png file.
        
        Parameters:
        timeSteps - number of steps done by the agent, the default is 100
        '''
        files = glob.glob(os.path.join('plane', '*')) # get access to all files in a folder
        for f in files:
            os.remove(f) # remove all files from folder before adding new ones
        
        self.timeSteps = timeSteps
        
        x = np.zeros(timeSteps + 1)
        y = np.zeros(timeSteps + 1)
        
        probA = 0
        probB = 0
        
        plt.rcParams['figure.figsize'] = [6, 6]
        plt.xlim(-self.planeSizeX/2, self.planeSizeX/2)
        plt.ylim(-self.planeSizeY/2, self.planeSizeY/2)
        plt.title("Agent's random walk on a plane")
        
        for t in range(timeSteps):
            angle = np.random.uniform(0, 2 * np.pi)
            while abs(x[t] + np.cos(angle)) > self.planeSizeX/2 or abs(y[t] + np.sin(angle)) > self.planeSizeY/2:
                angle = np.random.uniform(0, 2 * np.pi)
            x[t + 1] = x[t] + np.cos(angle)
            y[t + 1] = y[t] + np.sin(angle)
            
            if x[t] > 0:
                probA += 1
            if x[t] > 0 and y[t] > 0:
                probB += 1
                
            if toVideo == True:
                file = os.path.join('plane', f'move_{t}.png')
                
            if t == 0:
                plt.scatter(x[t], y[t], s = 30, c = 'green')
            if t == timeSteps - 1:
                plt.scatter(x[t], y[t], s = 30, c = 'red') 
            plt.scatter(x[t], y[t], s = 3, c = 'gray')
            plt.plot(x[:t+2], y[:t+2], c = 'black', ls = '--', lw = 0.5)
            plt.axvline(x = 0, c = 'gray', ls = ':', lw = 0.5)
            plt.axhline(y = 0, c = 'gray', ls = ':', lw = 0.5)
            
            if toVideo == True:
                plt.savefig(file)
            
        if showTraj == True:
            plt.show()
        else:
            plt.close()
        
        return probA/self.timeSteps, probB/self.timeSteps
    
    def graphARW(self, graphType, timeSteps = 100, p = 0.1, N = 10, showTraj = False, toVideo = False):
        '''
        sth
        '''
        
        if graphType == 'random':
            files = glob.glob(os.path.join('randomGraph', '*')) # get access to all files in a folder
            for f in files:
                os.remove(f) # remove all files from folder before adding new ones
            
            G = nx.Graph()
            
            random.seed(100)
            
            for i in range(N):
                G.add_node(i)

            for i in range(N):
                for j in range(N):
                    if i != j:
                        q = np.random.rand()
                        if q < p:
                            G.add_edge(i, j)
            
            nodes = list(G.nodes())
            origin = random.choice(nodes)
            walk = [origin]
            
            self.timeSteps = timeSteps
            
            if not nx.is_connected(G):
                nx.draw(G)
                return 'Graph is not connected. Try again'   
            
            for t in range(timeSteps):
                currentNode = walk[-1]
                neighbors = list(G.neighbors(currentNode))
                nextNode = random.choice(neighbors)
                walk.append(nextNode)
                
                #position = nx.spring_layout(G)
                
                pos = {}
                for i in range(N):
                    pos[i] = [np.cos(2*np.pi*i/N), np.sin(2*np.pi*i/N)]
            
                node_colors = ['red' if node == currentNode else 'gray' for node in G.nodes()]
                
                if toVideo == True:
                    file = os.path.join('randomGraph', f'move_{t}.png')

                nx.draw(G, pos, node_color = node_colors, with_labels=True)
                
                if toVideo == True:
                    plt.savefig(file)
            
            if showTraj == True:
                plt.show()
            else:
                plt.close()
    
    def createARWMovie(self, walkType, fps = 20):
        '''
        Creates a movie from saved images of a random walk of the agent.
        
        Parameters:
        walkType - type of walk which we want to create a move from, we can choose from
        lattice, plane, randomGraph
        fps - speed of the movie, the default is 20
        '''
        if walkType == 'lattice':
            filename = 'LatticeARW.mp4'
            folder = 'lattice'
        elif walkType == 'plane':
            filename = 'PlaneARW.mp4'
            folder = 'plane'
        elif walkType == 'randomGraph':
            filename = 'RandomGraphARW.mp4'
            folder = 'randomGraph'
            
        with imageio.get_writer(filename, mode = 'I', fps = fps) as w: # mode I means that function uses Images to create video
            for i in range(self.timeSteps):     
                file = os.path.join(folder, f'move_{i}.png')
                img = imageio.imread(file)
                w.append_data(img)

In [61]:
walk1 = randomWalk()
walk2 = randomWalk()
walk3 = randomWalk()

In [ ]:
walk1.latticeARW(timeSteps = 1000, showTraj = True)

In [ ]:
walk1.latticeARW(timeSteps = 100, toVideo = True)
walk1.createARWMovie('lattice')

In [ ]:
walk2.planeARW(timeSteps = 1000, showTraj = True)

In [ ]:
walk2.planeARW(timeSteps = 100, toVideo = True)
walk2.createARWMovie('plane')

In [64]:
walk3.graphARW('random', p = 0.4, timeSteps = 100, N = 20, toVideo = True)
walk3.createARWMovie('randomGraph', fps = 5)

In [ ]:
def probsPDFs(N, t):
    walk = randomWalk()
    probsA = []
    probsB = []
    for i in range(N):
        if i % 20 == 0:
            a, b = walk.planeARW(t, showTraj = True)
            probsA.append(a)
            probsB.append(b)
        else:
            a, b = walk.planeARW(t)
            probsA.append(a)
            probsB.append(b)
        
    plt.hist(probsA, bins = 'auto', density = True)
    plt.title("Histogram of probabilities that x > 0")
    plt.show()
    
    meanA = np.round(np.mean(probsA), 5)
    print(f'Probability that x > 0 is {meanA}')
    
    plt.hist(probsB, bins = 'auto', density = True)
    plt.title("Histogram of probabilities that x > 0 and y > 0")
    plt.show()
    
    meanB = np.round(np.mean(probsB), 5)
    print(f'Probability that x > 0 and y > 0 is {meanB}')

In [ ]:
probsPDFs(50, 1000)